In [2]:
# ===============================
# مشروع NLP عربي - Light10 Stemmer (محسن بدون ملفات)
# ===============================

# 1️⃣ استيراد المكتبات
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

# 2️⃣ تعريف البيانات العربية داخليًا
texts = [
    "والطلاب يدرسون اللغة العربية",
    "الكتابات العلمية مهمة للبحث",
    "أحب قراءة الكتب",
    "المدرسة تنظم الأنشطة الطلابية",
    "بحث الطلاب عن المعلومات مفيد",
    "أحب العلم والتعلم المستمر",
    "الطلاب يشاركون في المسابقات العلمية",
    "الكتب تحتوي على معلومات قيمة",
    "الدراسة تساعد في النجاح",
    "المعرفة مفتاح التقدم"
]

labels = [
    "تعليم", "بحث", "ثقافة", "تعليم", "بحث",
    "ثقافة", "تعليم", "ثقافة", "تعليم", "ثقافة"
]

# 3️⃣ تحسين Light10 Stemmer
prefixes = ["ال", "وال","و", "ف", "ب", "ك", "ل", "لل", "س"]
suffixes = ["ه", "ها", "ك", "ي", "هم", "ات", "ين", "ون", "ان", "ية", "ء", "نا", "تم"]

def light10_stem(word):
    # إزالة البادئات
    for p in prefixes:
        if word.startswith(p) and len(word) - len(p) >= 2:
            word = word[len(p):]
            break
    # إزالة اللاحقات
    for s in suffixes:
        if word.endswith(s) and len(word) - len(s) >= 2:
            word = word[:-len(s)]
            break
    return word

# 4️⃣ تنظيف النصوص ومعالجتها
def preprocess_text(text):
    if not text:
        return ""
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  # إزالة الرموز والأرقام
    text = re.sub(r"\s+", " ", text).strip()       # إزالة الفراغات الزائدة
    tokens = text.split()
    stems = [light10_stem(t) for t in tokens]
    return " ".join(stems)

processed_texts = [preprocess_text(t) for t in texts]

print("=== نصوص بعد المعالجة ===")
for original, processed in zip(texts, processed_texts):
    print(f"{original} --> {processed}")

# 5️⃣ تحويل النصوص إلى TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_texts)
y = labels

# 6️⃣ تقسيم البيانات وتدريب النموذج
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
model = MultinomialNB()
model.fit(X_train, y_train)

# 7️⃣ تقييم النموذج
y_pred = model.predict(X_test)
print("\n=== تقييم نموذج التصنيف ===")
print("الدقة:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))

# 8️⃣ وظيفة البحث/استرجاع النصوص الأكثر تشابهًا
def search_text(query, top_n=3):
    query_stem = preprocess_text(query)
    query_vec = vectorizer.transform([query_stem])
    similarity = cosine_similarity(query_vec, X)
    results = sorted(
        [(idx, score) for idx, score in enumerate(similarity[0])],
        key=lambda x: x[1],
        reverse=True
    )
    print(f"\n=== نتائج البحث عن: '{query}' ===")
    for idx, score in results[:top_n]:
        print(f"النص: {texts[idx]} | درجة التشابه: {score:.3f}")

# مثال على البحث
search_text("طلاب")


=== نصوص بعد المعالجة ===
والطلاب يدرسون اللغة العربية --> طلاب يدرس لغة عرب
الكتابات العلمية مهمة للبحث --> كتاب علم مهمة لبحث
أحب قراءة الكتب --> أحب قراءة كتب
المدرسة تنظم الأنشطة الطلابية --> مدرسة تنظم أنشطة طلاب
بحث الطلاب عن المعلومات مفيد --> حث طلاب عن معلوم مفيد
أحب العلم والتعلم المستمر --> أحب علم تعلم مستمر
الطلاب يشاركون في المسابقات العلمية --> طلاب يشارك في مسابق علم
الكتب تحتوي على معلومات قيمة --> كتب تحتو على معلوم قيمة
الدراسة تساعد في النجاح --> دراسة تساعد في نجاح
المعرفة مفتاح التقدم --> معرفة مفتاح تقدم

=== تقييم نموذج التصنيف ===
الدقة: 0.6666666666666666
              precision    recall  f1-score   support

         بحث       0.00      0.00      0.00         1
       تعليم       0.50      1.00      0.67         1
       ثقافة       1.00      1.00      1.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3


=== نتائج البحث عن: 'طلاب' ===